In [2]:
import os
import sys

work_dir = "/home/nlp/achimoa/workspace/hebrew_text_retrieval"
src_dir = os.path.join(work_dir, "src")
os.chdir(work_dir)

if src_dir not in sys.path:
    sys.path.append(src_dir)

In [3]:
from transformers import AutoConfig
from data.heq.heq_data import HeQDatasetBuilder, HeQTaskName
from model.dual_encoder.models import InfoNCEDualEncoder

In [4]:
heq_dataset_builder = HeQDatasetBuilder(task=HeQTaskName.QUESTION_DOC, decorate_with_task_tokens=False)
heq_dataset = heq_dataset_builder.build_dataset(filter_empty_answers=True, splits=['test'])
heq_dataset

url = https://raw.githubusercontent.com/NNLP-IL/Hebrew-Question-Answering-Dataset/refs/heads/main/data/data%20v1.1/test%20v1.1.json


Filter:   0%|          | 0/238 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['anchor_text', 'positive_text', 'index', 'paragraph_index', 'question_id', 'question', 'answer', 'context'],
        num_rows: 166
    })
})

In [5]:
from transformers import AutoConfig

# Set your checkpoint folder path
checkpoint_path = "/home/nlp/achimoa/workspace/hebrew_text_retrieval/outputs/models/dual_encoder/dual_encoder_infonce_heq/ckpt_20250522_1841_ep1-ba136000/checkpoint-4000"
checkpoint_path = "/home/nlp/achimoa/workspace/hebrew_text_retrieval/outputs/models/dual_encoder/dual_encoder_infonce_heq/ckpt_c20250603_1331_ep4-ba628000/checkpoint-4000"

# Load config if not already in your class
config = AutoConfig.from_pretrained(checkpoint_path)

# Now load your custom model with from_pretrained
model = InfoNCEDualEncoder.from_pretrained(checkpoint_path, config=config)


In [6]:
questions = [item["question"] for item in heq_dataset["test"]]
contexts = [item["context"] for item in heq_dataset["test"]]

In [7]:
from transformers import AutoTokenizer

model_name_or_path = "/home/nlp/achimoa/workspace/ModernBERT/hf/HebrewModernBERT/ModernBERT-Hebrew-base_20250603_1331"
tokenizer_q = AutoTokenizer.from_pretrained(model_name_or_path)  # or your actual model name
tokenizer_d = AutoTokenizer.from_pretrained(model_name_or_path)

q_batch = tokenizer_q(questions, padding=True, truncation=True, max_length=1024, return_tensors="pt")
d_batch = tokenizer_d(contexts, padding=True, truncation=True, max_length=1024, return_tensors="pt")


In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

with torch.no_grad():
    q_emb = model.encode(model.query_encoder, q_batch['input_ids'].to(device), q_batch['attention_mask'].to(device))
    d_emb = model.encode(model.doc_encoder, d_batch['input_ids'].to(device), d_batch['attention_mask'].to(device))
    # q_emb and a_emb are [num_samples, hidden_dim]


In [9]:
sim_matrix = torch.matmul(q_emb, d_emb.t())

In [10]:
import numpy as np
import nltk

def tlnls(gold, pred):
    gold_toks = gold.split()
    pred_toks = pred.split()
    lev = nltk.edit_distance(gold_toks, pred_toks)
    denom = max(len(gold_toks), len(pred_toks))
    return 1.0 if denom == 0 else 1 - lev / denom

sim_scores = sim_matrix.cpu().numpy()
N = sim_scores.shape[0]

ranks = []
tlnls_scores = []
for i in range(N):
    ranking = np.argsort(sim_scores[i])[::-1]
    rank = np.where(ranking == i)[0][0] + 1  # Gold doc is at position i
    ranks.append(rank)
    # TLNLS: Compare gold context to top-1 retrieved context
    pred_idx = ranking[0]
    gold_context = contexts[i]
    pred_context = contexts[pred_idx]
    tlnls_score = tlnls(gold_context, pred_context)
    tlnls_scores.append(tlnls_score)

accuracy = np.mean([r == 1 for r in ranks])
mrr = np.mean([1.0 / r for r in ranks])
mean_tlnls = np.mean(tlnls_scores)

print(f"Top-1 Accuracy: {accuracy:.4f}")
print(f"MRR: {mrr:.4f}")
print(f"Mean TLNLS (Top-1): {mean_tlnls:.4f}")

k = 5
recall_at_k = np.mean([r <= k for r in ranks])
print(f"Recall@{k}: {recall_at_k:.4f}")

k = 10
recall_at_k = np.mean([r <= k for r in ranks])
print(f"Recall@{k}: {recall_at_k:.4f}")



Top-1 Accuracy: 0.7892
MRR: 0.8452
Mean TLNLS (Top-1): 0.7936
Recall@5: 0.9157
Recall@10: 0.9639


In [11]:
import numpy as np
import random
random.seed(42)  # For reproducibility

# see `num_show` random questions
num_show = 5
top_k = 5

question_indices = random.sample(range(len(questions)), min(num_show, len(questions)))

for i in question_indices:
    print(f"\nQuestion [{i}]: {questions[i]}")
    print(f"Gold Context: {contexts[i]}")
    ranked_idxs = np.argsort(sim_scores[i])[::-1]
    print("Top predicted contexts:")
    for rank, idx in enumerate(ranked_idxs[:top_k]):
        mark = " <-- GOLD" if idx == i else ""
        print(f"  {rank+1}. {contexts[idx]}{mark} (score: {sim_scores[i][idx]:.4f})")
    gold_rank = np.where(ranked_idxs == i)[0][0] + 1
    print(f"Gold context is ranked at position: {gold_rank}")




Question [163]: מי קנתה את Protego?
Gold Context: זוהי הרכישה ה-16 של צ'ק פוינט מאז הקמתה, והרכישה הישראלית ה-6 שלה ב-3 השנים האחרונות, לה קדמו הרכישות של Protego ב-40 מיליון דולר, Cymplify שנרכש ב-6 מיליון דולר בלי מוצר, ForceNock שנרכש תוך שנתיים מהקמתו ועוד. גורמים שעימם שוחחנו בצ'ק פוינט העידו שישנן עוד רכישות באופק המיידי של החברה, אך סירבו להרחיב בנושא. "המסע שלנו להמציא מחדש את טכנולוגית הגנת הדוא"ל מתחיל פרק חדש וחשוב. יכולות ההגנה והמודיעין של צ'ק פוינט גבוהות מכלל המתחרות בשוק, והשילוב שלנו בתוכן יבטיח שנעצור התקפות דוא"ל מתוחכמות שכל האחרים מפספסים", מסר גיל פרידריך, מנכ"ל Avanan, בתגובה להודעת הרכישה. "בארגון גלובלי גדול ומוביל כמו צ'ק פוינט, אנחנו יכולים לייצר פתרון אחוד לכל סוגי הארגונים, מכל הגדלים והגיאוגרפיות, כך שכל מי שיצטרך הגנה טובה יותר על דוא"ל ואפליקציות שיתוף ארגוניות יוכל לקבל זאת".
Top predicted contexts:
  1. זוהי הרכישה ה-16 של צ'ק פוינט מאז הקמתה, והרכישה הישראלית ה-6 שלה ב-3 השנים האחרונות, לה קדמו הרכישות של Protego ב-40 מיליון דולר, Cymplify שנרכש ב-6 

In [12]:
# Let's say you want to see random contexts
context_indices = random.sample(range(len(contexts)), min(num_show, len(contexts)))

for j in context_indices:
    print(f"\nContext [{j}]: {contexts[j]}")
    print(f"Gold Question: {questions[j]}")
    ranked_q_idxs = np.argsort(sim_scores[:, j])[::-1]
    print("Top predicted questions:")
    for rank, idx in enumerate(ranked_q_idxs[:top_k]):
        mark = " <-- GOLD" if idx == j else ""
        print(f"  {rank+1}. {questions[idx]}{mark} (score: {sim_scores[idx][j]:.4f})")
    gold_rank = np.where(ranked_q_idxs == j)[0][0] + 1
    print(f"Gold question is ranked at position: {gold_rank}")



Context [57]: לקריאה תוקנו ברכות לפניה ולאחריה - "ברכות קריאת שמע", ובמשך הזמן הוצמדה לתפילת שחרית וערבית. אצל חז"ל מוזכרת גם "קריאת שמע על המיטה", בסמוך לשינה, ולפי דעתו של רש"י הייתה זו הקריאה של  בני ארץ ישראל אשר היו מתפללים תפילת ערבית לפני השקיעה, והיו מזכירים בה קריאת שמע רק כדי לעמוד מתוך דברי תורה. בהמשך התקבלה קריאה זו כקריאה עצמאית ונוספת, בסמוך אל ברכת המפיל לפני השינה, וכהגנה מפני סכנות הלילה. לדעת בעלי התוספות כבר מלכתחילה תוקנה כחלק מתפילת ערבית על שלשת פרשיותיה, ואילו בקריאת שמע שעל המיטה, נאמרת הפרשה הראשונה בלבד.
Gold Question: באיזו קריאה נאמרת הפרשה הראשונה בלבד?
Top predicted questions:
  1. כיצד הפיל הסטארטאפיסט את האתר של Post Israel? (score: 129.4996)
  2. באיזו קריאה נאמרת הפרשה הראשונה בלבד? <-- GOLD (score: 58.0524)
  3. באיזה מועד אומרים את התוספת "ברוך שם כבוד מלכותו לעולם ועד" בקול ולא בלחש? (score: 49.6880)
  4. איזה ספר שכתב הרב קוק נוהג הרב אבינר ללמד? (score: 36.7748)
  5. מהו מקום היוולדו של נתן אלתרמן? (score: 34.8643)
Gold question is ranked at pos

In [13]:
from collections import Counter
print(Counter(contexts).most_common(10))


[('הובר היה זה שעיצב את תוכנית הנאמנות והביטחון של הנשיא טרומן, ובדיקות הרקע של עובדי הממשלה בוצעו בידי סוכני ה-FBI. זו הייתה אחת המטלות המרכזיות של הארגון, והדבר הוביל להגדלת מספר הסוכנים מ-3,559 ב-1946 ל-7,029 ב-1952. תחושת האיום הקיצונית מפני הקומוניזם שחש הובר, וההתייחסות השמרנית פוליטית לעדויות שנגבו על ידי סוכניו הביאה לפיטוריהם של אלפי עובדי ממשלה. הובר עמד על כך שזהותם של המודיעים (שמסרו ידיעות על מי שנחשדו בקומוניזם) תישמר בסוד, ובשל כך רוב הנחקרים לא היו יכולים לחקור את מאשימיהם בחקירה נגדית ואף לא ידעו את זהותם. במקרים רבים לא נאמר להם כלל במה הם מואשמים.', 1), ('במהלך תקופת מקארתי הואשמו אלפים מאזרחי ארצות הברית בחברות במפלגה הקומוניסטית של ארצות הברית או באהדה כלפיה, והיו מטרה לחקירות נמרצות שנערכו על ידי גופים ממשלתיים או ועדות וסוכנויות אזרחיות. מושאי החקירות היו בעיקר עובדי ממשלת ארצות הברית, אנשים שהיו קשורים לתעשיית הבידור, מורים ופעילים בארגוני עובדים. לחשדות ניתנה אמינות, אף על פי שהעדויות שעל פיהן הועלו אותם חשדות היו לעיתים קרובות לא חד-משמעיות ואף מפוקפקות, הוערכ

In [14]:
import numpy as np

# sim_scores: similarity matrix (questions x contexts)
# questions: list of questions
# contexts: list of contexts

top_k = 5  # How many top predictions to print per error

errors = []

for i in range(len(questions)):
    ranked_idxs = np.argsort(sim_scores[i])[::-1]
    gold_rank = np.where(ranked_idxs == i)[0][0] + 1
    if gold_rank != 1:
        errors.append({
            "idx": i,
            "question": questions[i],
            "gold_context": contexts[i],
            "gold_rank": gold_rank,
            "top_predicted_indices": ranked_idxs[:top_k],
            "top_predicted_contexts": [contexts[j] for j in ranked_idxs[:top_k]],
            "top_predicted_scores": [sim_scores[i][j] for j in ranked_idxs[:top_k]],
        })

print(f"Found {len(errors)} errors out of {len(questions)}")

for error in errors:
    print(f"\nQuestion [{error['idx']}]: {error['question']}")
    print(f"Gold Context: {error['gold_context']}")
    print("Top predicted contexts:")
    for rank, (context, score) in enumerate(zip(error['top_predicted_contexts'], error['top_predicted_scores'])):
        mark = " <-- GOLD" if (contexts.index(context) == error['idx']) else ""
        print(f"  {rank+1}. {context}{mark} (score: {score:.4f})")
    print(f"Gold context is ranked at position: {error['gold_rank']}")


Found 35 errors out of 166

Question [9]: כיצד נקראה המשטרה החשאית של הצאר?
Gold Context: בהקשר זה יש להזכיר גם את הפרוטוקולים של זקני ציון. מסמך זה, שזוייף כנראה על ידי האוכרנה, המשטרה החשאית של הצאר, (על בסיס מאמר סאטירי, שהוצא מהקשרו, ושנועד לתקוף את משטרו של נפוליון השלישי בצרפת), זכה לתפוצה של ממש באירופה של סוף המאה ה-19 ובתחילת המאה ה-20. במסמך, המתעד פגישה כביכול של "זקני ציון" בבית הקברות היהודי בפראג, מתואר קשר יהודי להשתלט על העולם. על אף שזיוף המסמך בולט לעין, והקשר המתואר בו מופרך מעיקרו, הפך הספר לאבן הבסיס של האידאולוגיה האנטישמית ברחבי העולם. ברור כי מעולם לא התקיימו הפגישות המתוארות ב"פרוטקולים" ואף לא היה קיים ארגון בשם "זקני ציון" מעולם.
Top predicted contexts:
  1. העיר נוסדה ב-27 במאי 1703 על ידי הצאר פיוטר הגדול כ"חלון לאירופה", ושימשה כבירת האימפריה הרוסית בשנים 1721–1917, למעט השנים 1728–1730 שבהן מוסקבה שימשה כעיר הבירה. היא חדלה לשמש עיר הבירה לאחר מהפכת אוקטובר, כאשר הבירה שבה למוסקבה והעיר שינתה את שמה ללנינגרד. אוכלוסיית העיר מונה מעל 5 מיליון תושבים (2018)